### gensim 关键概念

gensim 通过检查训练文集中统计意义上的共现模式(co-occurance pattern)，来分析文档的语义结构。 它主要是基于非监督学习方法的。

一旦这些统计模式建立，任何普通的文档都可以用这种模式来表达，从而可以计算各种语义相似性(word,phrase,sentene,document)。

Gensim is a free Python library designed to automatically extract semantic topics from documents, as efficiently (computer-wise) and painlessly (human-wise) as possible.

Gensim is designed to process raw, unstructured digital texts (“plain text”).

The algorithms in Gensim, such as Word2Vec, FastText, Latent Semantic Analysis (LSI, LSA, see LsiModel), Latent Dirichlet Allocation (LDA, see LdaModel) etc, automatically discover the semantic structure of documents by examining statistical co-occurrence patterns within a corpus of training documents. These algorithms are unsupervised, which means no human input is necessary – you only need a corpus of plain text documents.

Once these statistical patterns are found, any plain text documents (sentence, phrase, word…) can be succinctly expressed in the new, semantic representation and queried for topical similarity against other documents (words, phrases…).


corpora 语料：可以通过字典将原始的文本转换成词袋模型  bag of words.  用稀疏矩阵表示。

models 模型：不同的模型可以将corpus 转换成其它表示，如 Tfidf, word2vec，LDA等等。

similarities  相似度: 基于模型转换后的文档表示，可以计算不同文档之间的相似度。


### Tfidf模型示范

In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
from gensim import corpora, models, similarities
corpus = [[(0, 1.0), (1, 1.0), (2, 1.0)],
          [(2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0), (6, 1.0), (8, 1.0)],
          [(1, 1.0), (3, 1.0), (4, 1.0), (7, 1.0)],
          [(0, 1.0), (4, 2.0), (7, 1.0)],
          [(3, 1.0), (5, 1.0), (6, 1.0)],
          [(9, 1.0)],
          [(9, 1.0), (10, 1.0)],
          [(9, 1.0), (10, 1.0), (11, 1.0)],
          [(8, 1.0), (10, 1.0), (11, 1.0)]]


C:\ProgrameFiles\Anaconda\envs\py3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2019-01-21 14:37:05,235 : INFO : 'pattern' package not found; tag filters are not available for English


In [19]:
import numpy as np 
import gensim
tfidf = models.TfidfModel(corpus)

2019-01-21 15:02:22,806 : INFO : collecting document frequencies
2019-01-21 15:02:22,813 : INFO : PROGRESS: processing document #0
2019-01-21 15:02:22,816 : INFO : calculating IDF weights for 9 documents and 11 features (28 matrix non-zeros)


In [20]:
vec = [(0, 1), (4, 1)]
print(tfidf[vec])

[(0, 0.8075244024440723), (4, 0.5898341626740045)]


In [ ]:
index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features=12)

In [ ]:
sims = index[tfidf[vec]]
print(list(enumerate(sims)))

In [ ]:
sims

In [ ]:
vec

In [ ]:
from gensim import matutils

matutils.corpus2dense([enumerate(sims)],num_terms= 50 )

### LSI示范

In [10]:
from gensim import corpora,models,similarities
documents = ["Human machine interface for lab abc computer applications",
             "A survey of user opinion of computer system response time",
             "The EPS user interface management system",
             "System and human system engineering testing of EPS",
             "Relation of user perceived response time to error measurement",
             "The generation of random binary unordered trees",
             "The intersection graph of paths in trees",
             "Graph minors IV Widths of trees and well quasi ordering",
             "Graph minors A survey"]

In [11]:
# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in documents]

# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
        
texts = [[token for token in text if frequency[token] > 1]
         for text in texts]

from pprint import pprint  # pretty-printer
pprint(texts)

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


In [12]:
dictionary = corpora.Dictionary(texts)
dictionary.save('deerwester.dict')  # store the dictionary, for future reference
print(dictionary)


2019-01-14 11:13:58,015 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-01-14 11:13:58,025 : INFO : built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)
2019-01-14 11:13:58,029 : INFO : saving Dictionary object under deerwester.dict, separately None
2019-01-14 11:13:58,032 : INFO : saved deerwester.dict


Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...)


In [13]:
print(dictionary.token2id)

{'computer': 0, 'human': 1, 'interface': 2, 'response': 3, 'survey': 4, 'system': 5, 'time': 6, 'user': 7, 'eps': 8, 'trees': 9, 'graph': 10, 'minors': 11}


In [ ]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('deerwester.mm', corpus)  # store to disk, for later use
pprint(corpus)

In [ ]:
# extract 400 LSI topics; use the default one-pass algorithm
lsi = models.lsimodel.LsiModel(corpus=corpus, id2word=dictionary, num_topics=5)
# print the most contributing words (both positively and negatively) for each of the first ten topics
lsi.print_topics(4)

In [ ]:
new_text = ['human','computer','user']
new_corpus = dictionary.doc2bow(new_text) 
lsi.add_documents([new_corpus])

In [ ]:
lsi[new_corpus]

In [ ]:
index = similarities.SparseMatrixSimilarity(lsi[corpus], num_features=12)

In [ ]:
sims = index[lsi[new_corpus]] 

In [ ]:
print(list(enumerate(sims)))

In [ ]:
list(lsi[corpus])

### LDA示范

In [1]:
from gensim import corpora, models,similarities

C:\ProgrameFiles\Anaconda\envs\py3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
import jieba
import numpy as np 
import pandas as pd 


dfapp = pd.read_csv('zz_apps_info_22k',sep = '\001',encoding = 'utf-8')
app_name = dfapp['app_name'].values.tolist()

jieba.load_userdict(app_name)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\liangyun\AppData\Local\Temp\jieba.cache
Loading model cost 0.838 seconds.
Prefix dict has been built succesfully.


In [3]:
app_info = dfapp['app_info'].values.tolist()

In [4]:
app_info_segs = [jieba.lcut(x) for x in app_info]

In [6]:
from six import iteritems
# collect statistics about all tokens
dictionary = corpora.Dictionary(app_info_segs)
# remove stop words and words that appear only once
once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq == 1]
dictionary.filter_tokens(once_ids)  # remove stop words and words that appear only once
dictionary.filter_extremes()
dictionary.compactify()  # remove gaps in id sequence after words that were removed


In [7]:
class MyCorpus(object):
    def __iter__(self):
        for seg in app_info_segs:
            # assume there's one document per line, tokens separated by whitespace
            yield dictionary.doc2bow(seg)

In [8]:
corpus_memory_friendly = MyCorpus()  # doesn't load the corpus into memory!
print(corpus_memory_friendly)

In [12]:
lda = models.ldamodel.LdaModel(corpus=corpus_memory_friendly,
                              id2word=dictionary, num_topics=100,
                              update_every=100, chunksize=1000, passes=10)

C:\ProgrameFiles\Anaconda\envs\py3\lib\site-packages\gensim\models\ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [13]:
lda.print_topics(100)

[(0,
  '0.078*" " + 0.021*"：" + 0.015*"." + 0.014*"！" + 0.009*"【" + 0.009*"】" + 0.006*"；" + 0.006*"-" + 0.006*"1" + 0.006*"学习"'),
 (1,
  '0.088*" " + 0.017*"." + 0.013*"你" + 0.013*"运动" + 0.011*"手机" + 0.010*"," + 0.009*"：" + 0.008*"支持" + 0.008*")" + 0.007*"更"'),
 (2,
  '0.021*"农业" + 0.015*"红包" + 0.013*"：" + 0.011*"身体" + 0.010*"通过" + 0.010*"和" + 0.010*" " + 0.010*"种植" + 0.009*"账号" + 0.008*"农民"'),
 (3,
  '0.035*"你" + 0.011*"！" + 0.011*"在" + 0.009*"？" + 0.009*"主播" + 0.009*"直播" + 0.008*"有" + 0.007*"和" + 0.007*"游戏" + 0.007*"自己"'),
 (4,
  '0.025*"VR" + 0.016*" " + 0.015*"：" + 0.014*"你" + 0.010*"-" + 0.009*"全景" + 0.008*"有" + 0.008*"体验" + 0.008*"更" + 0.007*"您"'),
 (5,
  '0.048*"装修" + 0.013*"您" + 0.012*"【" + 0.011*"】" + 0.011*"：" + 0.010*"." + 0.009*"！" + 0.009*"流量" + 0.008*"你" + 0.007*"让"'),
 (6,
  '0.018*"：" + 0.016*"服务" + 0.011*"等" + 0.009*"】" + 0.009*"和" + 0.009*"【" + 0.009*"为" + 0.009*" " + 0.008*"管理" + 0.008*"提供"'),
 (7,
  '0.077*" " + 0.017*"." + 0.015*"：" + 0.011*"支持" + 0.009*"和" + 0.007

In [32]:
from gensim import matutils

features  = matutils.corpus2dense(list(lda[corpus_memory_friendly]),num_terms= 100 ).T


In [34]:
np.sum(features,axis = 1)

array([0.996332  , 0.9903158 , 0.98344827, ..., 0.98099995, 0.97394735,
       0.97869563], dtype=float32)

### word2vec示范

word词的表示的最基础方法是one-hot Encoding，有时候也可以叫做one-hot Embedding。
 
man :  [1,0,0,0]

woman: [0,1,0,0]

boy :  [0,0,1,0]

girl:  [0,0,0,1]

但是这种表示过于稀疏，并且没有体现词与词之间的语义关系。
考虑以下两个维度的表示, gender和age

man : [male,adult]  -> [1,1]

woman: [female,adult] ->[0,1]

boy: [male,child] -> [1,0]

girl: [female,child] ->[0,0]

这种把非结构化数据里的元素用一个低维空间的向量来表示的方法叫做Embedding,

即把高维数据嵌入到低维空间。

但是不可能对所有word都这样人工构建Embedding,一种非常有效的方法是训练一个神经网络来得到。

通过给神经网络一个相关的任务，即由一个词预测它周围的词出现的概率，来得到词向量。

In [263]:
import jieba
import numpy as np 
import pandas as pd 

dfapp = pd.read_csv('zz_apps_info_22k',sep = '\001',encoding = 'utf-8')
app_name = dfapp['app_name'].values.tolist()

jieba.load_userdict(app_name)

app_info = dfapp['app_info'].values.tolist()

app_info_segs = [jieba.lcut(x) for x in app_info]

In [264]:
import logging
import os
from gensim.models import word2vec

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
#sentences = word2vec.LineSentence('./in_the_name_of_people_segment.txt') 

model = word2vec.Word2Vec(app_info_segs, hs=1,min_count=1,window=3,size=100)  

2019-01-14 17:24:34,822 : INFO : collecting all words and their counts
2019-01-14 17:24:34,831 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-01-14 17:24:35,199 : INFO : PROGRESS: at sentence #10000, processed 1642104 words, keeping 71862 word types
2019-01-14 17:24:35,499 : INFO : PROGRESS: at sentence #20000, processed 3142507 words, keeping 110306 word types
2019-01-14 17:24:35,572 : INFO : collected 117183 word types from a corpus of 3452173 raw words and 22262 sentences
2019-01-14 17:24:35,573 : INFO : Loading a fresh vocabulary
2019-01-14 17:24:36,104 : INFO : effective_min_count=1 retains 117183 unique words (100% of original 117183, drops 0)
2019-01-14 17:24:36,104 : INFO : effective_min_count=1 leaves 3452173 word corpus (100% of original 3452173, drops 0)
2019-01-14 17:24:36,496 : INFO : deleting the raw counts dictionary of 117183 items
2019-01-14 17:24:36,498 : INFO : sample=0.001 downsamples 31 most-common words
2019-01-14 17:24:36,499 : IN

In [276]:
model.wv.save_word2vec_format('appwordvec.txt',binary = False)

2019-01-14 18:13:11,544 : INFO : storing 117183x100 projection weights into appwordvec.txt


In [266]:
model.wv['融360']   # 查看词向量

array([-0.16399802, -0.64898807, -0.47534418, -0.5747225 ,  1.0914922 ,
       -0.62572366, -0.12807035, -0.46989697,  0.16053031,  1.1880977 ,
        0.5412748 , -0.31963104,  0.6387269 ,  0.46497753,  0.3419621 ,
        0.0715302 ,  0.2364061 , -0.02447075,  0.12523286, -0.1541518 ,
        0.3029079 ,  0.81672823, -0.14660323,  0.12207115, -0.31254426,
        1.1355394 , -0.18901029,  1.217527  , -0.46575525, -0.19007757,
        0.5349934 , -0.6021949 , -0.59060365,  0.30716202,  0.13562287,
        0.30115876,  0.08360162, -0.57765454,  0.567858  ,  0.2865388 ,
        0.05132287, -1.0141915 ,  0.29006585,  0.15064555,  1.1122638 ,
       -0.04659883, -1.0168066 , -0.2635237 , -0.5632418 , -0.13803859,
        0.3391766 , -0.6414495 ,  0.36325642,  0.1805825 ,  0.21871212,
        0.20217142, -0.43761793,  0.07297394, -0.10965775,  0.15736799,
        0.5622868 , -0.77992904,  0.5695759 ,  0.5154757 , -0.6231745 ,
        0.4218748 , -0.28670028, -0.61878514, -0.43199375,  0.25

In [267]:
model.wv.most_similar(['app'],topn = 20)

2019-01-14 18:11:57,257 : INFO : precomputing L2-norms of word weight vectors


[('App', 0.8908683061599731),
 ('APP', 0.8786344528198242),
 ('软件', 0.7927603721618652),
 ('应用', 0.7687703371047974),
 ('应用软件', 0.7153639793395996),
 ('客户端', 0.6696969866752625),
 ('平台', 0.6450727581977844),
 ('应用程序', 0.6196110248565674),
 ('神器', 0.5724780559539795),
 ('工具', 0.5697617530822754),
 ('手机软件', 0.5479514598846436),
 ('网站', 0.5304775238037109),
 ('程序', 0.5283219814300537),
 ('程式', 0.5237851738929749),
 ('定位模拟器', 0.5227001905441284),
 ('该软件', 0.5180087089538574),
 ('软件平台', 0.5179553031921387),
 ('手机', 0.5156241059303284),
 ('产品', 0.5153759717941284),
 ('有鱼记账', 0.5079294443130493)]

In [268]:
model.wv.most_similar(positive = ['急速','快捷'],negative = ['周期长'],topn = 10)

[('极速', 0.6685068607330322),
 ('快速', 0.630969762802124),
 ('闪电', 0.5957939624786377),
 ('便捷', 0.5763149857521057),
 ('速度快', 0.5684813261032104),
 ('高效', 0.5674407482147217),
 ('全自动', 0.5223191976547241),
 ('高价', 0.5196690559387207),
 ('更快', 0.5194609761238098),
 ('两证', 0.5183494091033936)]

In [271]:
model.wv.similarity('闪电', '快捷')

0.3643375

In [274]:
model.wv.n_similarity(['借款','贷款'],['下单','放款'])

0.65181714

In [8]:
model.wv.save('appinfo.wv')

2019-01-11 19:06:16,905 : INFO : saving Word2VecKeyedVectors object under appinfo.wv, separately None
2019-01-11 19:06:16,921 : INFO : storing np array 'vectors' to appinfo.wv.vectors.npy
2019-01-11 19:06:17,042 : INFO : not storing attribute vectors_norm
2019-01-11 19:06:18,573 : INFO : saved appinfo.wv


## doc2vec示范

doc2vec是word2vec的推广，有时候也叫作paragraph2vec,可以把一篇文档转换成vec。

其主要思想是在预测某处的单词时，不仅考虑其附近的单词上下文，还将其所在的文档的完整上下文作为一个输入信息。

此外,doc2vec还能够将和文档关联的其他监督类的标签加入训练中，并帮助提升模型效果。

#### 英文doc2vec示范

In [46]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(common_texts)]
model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)

2019-01-14 14:57:00,057 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2019-01-14 14:57:00,063 : INFO : collecting all words and their counts
2019-01-14 14:57:00,069 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2019-01-14 14:57:00,074 : INFO : collected 12 word types and 9 unique tags from a corpus of 9 examples and 29 words
2019-01-14 14:57:00,079 : INFO : Loading a fresh vocabulary
2019-01-14 14:57:00,081 : INFO : effective_min_count=1 retains 12 unique words (100% of original 12, drops 0)
2019-01-14 14:57:00,084 : INFO : effective_min_count=1 leaves 29 word corpus (100% of original 29, drops 0)
2019-01-14 14:57:00,087 : INFO : deleting the raw counts dictionary of 12 items
2019-01-14 14:57:00,089 : INFO : sample=0.001 downsamples 12 most-common words
2019-01-14 14:57:00,091 : INFO : downsampling leaves estimated 3 word corpus (12.1% of prior 29)
2019-01-14 14:57:00,093 : INFO : estimated required memory for 12 wo

In [43]:
documents

[TaggedDocument(words=['human', 'interface', 'computer'], tags=[0]),
 TaggedDocument(words=['survey', 'user', 'computer', 'system', 'response', 'time'], tags=[1]),
 TaggedDocument(words=['eps', 'user', 'interface', 'system'], tags=[2]),
 TaggedDocument(words=['system', 'human', 'system', 'eps'], tags=[3]),
 TaggedDocument(words=['user', 'response', 'time'], tags=[4]),
 TaggedDocument(words=['trees'], tags=[5]),
 TaggedDocument(words=['graph', 'trees'], tags=[6]),
 TaggedDocument(words=['graph', 'minors', 'trees'], tags=[7]),
 TaggedDocument(words=['graph', 'minors', 'survey'], tags=[8])]

In [73]:
#保存模型
model.save('all_model.doc2vec')

2019-01-14 15:10:41,898 : INFO : saving Doc2Vec object under all_model.doc2vec, separately None
2019-01-14 15:10:41,901 : INFO : saved all_model.doc2vec


In [81]:
# 保存文档向量
# save vectors
with open("all_vector.txt","w") as f:
    for num in range(0,len(documents)):
        docvec =model.docvecs[num]
        strvec = ' '.join([str(x) for x in docvec])
        f.write(strvec + '\n')
        print(num)
        print(docvec)

0
[ 0.03696746 -0.0187734   0.0083677   0.01484122  0.08131897]
1
[-0.05001155  0.04985023 -0.04786721 -0.05968755  0.0089878 ]
2
[-0.0251735  -0.06244595  0.0177445   0.09957398  0.03126098]
3
[ 0.04499181 -0.00525833  0.03086685 -0.0447739   0.00595002]
4
[-0.06912751 -0.01017872  0.01550646 -0.00949226 -0.0926289 ]
5
[-0.09529006 -0.05795394  0.02646667 -0.06973509 -0.02818455]
6
[ 0.01666221  0.09269849 -0.0726633  -0.09686001  0.01582416]
7
[ 0.04950825 -0.02783915  0.07833885 -0.0219612  -0.01475391]
8
[-0.05964534  0.01433995  0.04812507  0.01526107  0.02440032]


In [54]:
# 计算新文档的向量
model.infer_vector(["system", "response"])

array([ 0.01761255, -0.00106964,  0.01830512,  0.08613501,  0.05934137],
      dtype=float32)

In [67]:
# 计算与第1篇文档最相似的文档
model.docvecs.most_similar(0,topn=3)

[(2, 0.37856197357177734), (3, 0.2789281904697418), (7, 0.15917153656482697)]

In [71]:
# 计算2篇文档的相似度
model.docvecs.similarity(0,2)

0.378562

#### 中文doc2vec示范

In [233]:
import gensim, logging
import os
logging.basicConfig(format = '%(asctime)s : %(levelname)s : %(message)s', level = logging.INFO)

In [234]:
import jieba
import numpy as np 
import pandas as pd 

dfapp = pd.read_csv('zz_apps_info_22k',sep = '\001',encoding = 'utf-8')
app_name = dfapp['app_name'].values.tolist()

jieba.load_userdict(app_name)

app_info = dfapp['app_info'].values.tolist()
#app_info_segs = [jieba.lcut(x) for x in app_info]

alldocs,count = [],0

with open('cutted_app_info.txt','w',encoding = 'utf-8') as f:
    for info in app_info:
        segs = ' '.join(jieba.lcut(info))
        f.write(segs + '\n')        
        alldocs.append(gensim.models.doc2vec.TaggedDocument(segs,count)) # for the sake of check, can be removed
        count+=1 # for the sake of check, can be removed

In [235]:
# 训练模型
sentences = gensim.models.doc2vec.TaggedLineDocument('cutted_app_info.txt')
model = gensim.models.Doc2Vec(sentences, size = 20, window = 3)

2019-01-14 16:48:00,112 : INFO : collecting all words and their counts
2019-01-14 16:48:00,114 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2019-01-14 16:48:00,682 : INFO : PROGRESS: at example #10000, processed 1591828 words (2814869/s), 71854 word types, 10000 tags
2019-01-14 16:48:01,128 : INFO : PROGRESS: at example #20000, processed 2873143 words (2873697/s), 106226 word types, 20000 tags
2019-01-14 16:48:01,301 : INFO : collected 117173 word types and 23655 unique tags from a corpus of 23655 examples and 3341352 words
2019-01-14 16:48:01,302 : INFO : Loading a fresh vocabulary
2019-01-14 16:48:01,410 : INFO : effective_min_count=5 retains 28016 unique words (23% of original 117173, drops 89157)
2019-01-14 16:48:01,412 : INFO : effective_min_count=5 leaves 3199012 word corpus (95% of original 3341352, drops 142340)
2019-01-14 16:48:01,550 : INFO : deleting the raw counts dictionary of 117173 items
2019-01-14 16:48:01,554 : INFO : sample=0.001 dow

In [239]:
model.wv.vocab

{'融360': <gensim.models.keyedvectors.Vocab at 0x30837e80>,
 '—': <gensim.models.keyedvectors.Vocab at 0x30837c50>,
 '您': <gensim.models.keyedvectors.Vocab at 0x30837c88>,
 '随身': <gensim.models.keyedvectors.Vocab at 0x308376d8>,
 '的': <gensim.models.keyedvectors.Vocab at 0x30837f60>,
 '金融': <gensim.models.keyedvectors.Vocab at 0x30837b00>,
 '专家': <gensim.models.keyedvectors.Vocab at 0x30837d30>,
 '为什么': <gensim.models.keyedvectors.Vocab at 0x30838048>,
 '要': <gensim.models.keyedvectors.Vocab at 0x308380f0>,
 '使用': <gensim.models.keyedvectors.Vocab at 0x30838128>,
 '【': <gensim.models.keyedvectors.Vocab at 0x308387b8>,
 '】': <gensim.models.keyedvectors.Vocab at 0x30838828>,
 '？': <gensim.models.keyedvectors.Vocab at 0x308385c0>,
 '下述': <gensim.models.keyedvectors.Vocab at 0x30838198>,
 '内容': <gensim.models.keyedvectors.Vocab at 0x30838208>,
 '不': <gensim.models.keyedvectors.Vocab at 0x30838278>,
 '吹牛': <gensim.models.keyedvectors.Vocab at 0x308382e8>,
 '。': <gensim.models.keyedvectors.Vo

In [236]:
#model.save('review_pure_text_model.txt')
print(len(model.docvecs))

23655


In [138]:
# 保存文档向量
# save vectors
with open("doc_vector.txt","w") as f:
    for num in range(0,len(model.docvecs)):
        docvec =model.docvecs[num]
        strvec = ' '.join([str(x) for x in docvec])
        f.write(strvec + '\n')

In [140]:
dfapp['app_name'].head(10)

0        融360
1          微信
2          设置
3         支付宝
4          信息
5          QQ
6         指南针
7        手机淘宝
8         云服务
9    WiFi万能钥匙
Name: app_name, dtype: object

In [141]:
# 能够获取wordvec信息
model.wv.similar_by_word('蚂蚁')

2019-01-14 16:04:10,567 : INFO : precomputing L2-norms of word weight vectors


[('金服', 0.8884478807449341),
 ('苏宁', 0.8769680261611938),
 ('亲亲', 0.8668301105499268),
 ('我来贷', 0.8627084493637085),
 ('兴业银行', 0.8612933158874512),
 ('小说网', 0.8539562225341797),
 ('融360', 0.8534164428710938),
 ('平安', 0.8503285050392151),
 ('中信银行', 0.848244845867157),
 ('宜人', 0.8451166749000549)]

In [142]:
# 
model.wv.most_similar('金融')

[('互联网', 0.8961374759674072),
 ('金融公司', 0.8875133395195007),
 ('金融服务', 0.8836145997047424),
 ('电商', 0.8568190336227417),
 ('供应链', 0.8504671454429626),
 ('服务商', 0.8414190411567688),
 ('综合', 0.8371357917785645),
 ('运营', 0.8332608342170715),
 ('金融信息', 0.8317973613739014),
 ('创新型', 0.8296557068824768)]

In [143]:
model.wv.similarity('闪电', '快捷')

0.5296482

In [144]:
# 获取docvec信息
model.docvecs[1]

array([ 0.26246032,  0.25317916,  0.10804079, -0.28541595, -0.03877484,
       -0.17073359,  0.06715155,  0.10488174,  0.45085293, -0.04337186,
        0.20255078,  0.02666478, -0.35303056, -0.00325981,  0.13049288,
       -0.3592315 , -0.11136763,  0.10300702,  0.22605814, -0.23496947],
      dtype=float32)

In [145]:
alldocs[1].words

'1 . 可以 发 语音 、 文字 消息 、 表情 、 图片 、 视频 30M 流量 可以 收发 上 千条 语音 ， 省电省 流量 2 . 朋友圈 ， 跟 朋友 们 分享 生活 点滴 3 . 摇一摇 、 查看 附近的人 ， 世界 不再 有 陌生人 4 . 扫一扫 ， 可以 扫 商品 条码 、 图书 封面 、 CD 封面 ， 甚至 扫描 英文单词 来 翻译成 中文 5 . 公众 帐号 ， 用 微信 关注 明星 、 看 新闻 、 设 提醒 6 . 游戏中心 ， 和 朋友 们 一起 玩游戏 7 . 表情 商店 ， 有趣 好玩 的 表情 在 这里 特别 说明 ： 微信 只 消耗 网络流量 ， 不 产生 短信 电话 费用'

In [161]:
# 计算新文档的向量
inferred_docvec = model.infer_vector(alldocs[1].words)
print('%s:\n %s' % (model, model.docvecs.most_similar([inferred_docvec], topn=10)))

Doc2Vec(dm/m,d20,n5,w3,mc5,s0.001,t3):
 [(17043, 0.8203989863395691), (10917, 0.7990165948867798), (16571, 0.7887634038925171), (8152, 0.7831325531005859), (3550, 0.7769162058830261), (13602, 0.7696261405944824), (6090, 0.7670239806175232), (9233, 0.765042781829834), (18120, 0.7551760673522949), (9855, 0.750206708908081)]


In [147]:
# 计算与第1篇文档最相似的文档
model.docvecs.most_similar(1,topn=10)

[(18808, 0.8535903096199036),
 (11547, 0.8345423936843872),
 (11367, 0.7948375940322876),
 (4428, 0.7887148261070251),
 (15580, 0.786209225654602),
 (13601, 0.768325686454773),
 (11214, 0.7595034241676331),
 (7430, 0.7570552229881287),
 (21464, 0.7564826011657715),
 (47, 0.7517760992050171)]

In [114]:
dfapp.loc[1,'app_info']

'1.可以发语音、文字消息、表情、图片、视频30M流量可以收发上千条语音，省电省流量2.朋友圈，跟朋友们分享生活点滴3.摇一摇、查看附近的人，世界不再有陌生人4.扫一扫，可以扫商品条码、图书封面、CD封面，甚至扫描英文单词来翻译成中文5.公众帐号，用微信关注明星、看新闻、设提醒6.游戏中心，和朋友们一起玩游戏7.表情商店，有趣好玩的表情在这里特别说明：微信只消耗网络流量，不产生短信电话费用'

In [116]:
dfapp.loc[11547,'app_info']

'Coolhear 3D是一款能够带给您全新3D体验的音乐播放器！360度全范围任意3D编曲，3D音乐与身体实时互动，奇幻的3D世界近在耳边！不同场景内容，午休、运动、旅行、夜店、学习、趣味、自然。。。有你想不到，木有你不需要！更多玩法，更多惊喜，等待您来体验！'

In [151]:
dfapp.loc[112,'app_info']

'招联金融APP5.0，与你为5 一起体验智能无息时代招联金融——信用贷款免息分期借钱：在线申请！1分钟放款！额度高达20万！日利率低至0.029%！【平台介绍】招联金融：由招商银行与中国联通共同组建，银监会消费金融持牌机构。为您提供银行级安全、专业、智能、便捷的现金借贷、分期消费、免息购物服务。【产品简介】● 现金借贷：即“招联好期贷”2.0，为您提供专业的贷款服务超低利率，按日计息，随借随还，智能匹配最高额度新用户50000元免息借、提额、办理信用卡、线下分期消费● 免息商城：使用招联金融信用额度，先消费、后还款全场0首付免息，5折包邮，千家品牌直供，更有天天秒杀、0元试用、1元拼精品好物等你来抢● 精选首页：每日为您选择最适合您的内容和功能瓜分1亿元减息金、还款随机立减最高88元、12期免息购物、专享每周提额、会员专属福利社、天天领现金、1元秒杀精选商品等APP专属特权及活动邀你来参与'

In [175]:
# 计算2篇文档的相似度

#微信和QQ的相似度
model.docvecs.similarity(1,5)

0.47017232

In [174]:
#微信和支付宝相似度
model.docvecs.similarity(1,3)

0.3176023

In [214]:
alldocs[0]

TaggedDocument(words='融360   — —   您 随身 的 金融 专家 为什么 要 使用 【 融360 】 ？ 下述 内容 不 吹牛 。 1 .   解决 【 贷款 】 难题 特点 ： -   全面解决 ： 个人消费 贷 、 企业 经营 贷 、 车房 装修 贷 、 出行 旅游 贷 、 学习 进修 贷 。 -   放款 效率 ： 仅 需 一天 即可 放款 。 -   利率 优势 ： 年化 费率 低至 6% ， 还款 期限 3 个 月 起 。 -   服务 优质 ： 我们 深知 您 的 难 和 急 ， 暖心 的 服务 是 我们 理应 为 您 真情 奉献 的 。 2 .   全面 的 【 信用卡 资讯 】 特点 ： -   优惠 资讯 ： 各行 信用卡 优惠 资讯 实时 提供 ， 吃喝玩乐 省 不停 。 -   信用卡 论坛 ： 信用卡 达人 和 小白 都 爱 逛得 专属 论坛 。 3 .   提供 【 金融信息 】 大全 特点 ： -   金融信息 ： 涵盖 各种 办卡 技巧 、 贷款 技巧 、 经济 政策 、 金融 趣谈 。 融360 公司 介绍 融360 ， 成立 于 2011 年 10 月 ， 是 中国 值得 信赖 的 金融 垂直 搜索 平台 。 融360 致力于 为 个人 消费者 和 微小 企业 提供 金融 产品 的 搜索 、 推荐 和 申请 服务 ， 业务范围 涵盖 贷款 、 信用卡 。 此外 ， 融360 还 免费 为 用户 提供 便捷 、 划算 、 安全 的 金融信息 。 融360 目前 已 完成 D 轮 融资 ， 估值 超 10 亿美元 ， 步入 独角兽 俱乐部 。 融360 旗下 简普 科技 2017 年 11 月 在 美国 纽交所 上市 。 联系 我们   — — 【 官方网站 】 http : / / m . rong360 . com 【 官方 微信 公众 号 】 融360 【 客服 电话 】 1010 0360 或 4008 - 905 - 360', tags=0)

#### 加入监督tag示范

In [176]:
import gensim, logging
import os
logging.basicConfig(format = '%(asctime)s : %(levelname)s : %(message)s', level = logging.INFO)

In [177]:
import jieba
import numpy as np 
import pandas as pd 

dfapp = pd.read_csv('zz_apps_info_22k',sep = '\001',encoding = 'utf-8')
app_name = dfapp['app_name'].values.tolist()

jieba.load_userdict(app_name)

app_info = dfapp['app_info'].values.tolist()
#app_info_segs = [jieba.lcut(x) for x in app_info]

alldocs,count = [],0

with open('cutted_app_info.txt','w',encoding = 'utf-8') as f:
    for info in app_info:
        segs = ' '.join(jieba.lcut(info))
        f.write(segs + '\n')        
        alldocs.append(gensim.models.doc2vec.TaggedDocument(segs,count)) # for the sake of check, can be removed
        count+=1 # for the sake of check, can be removed

In [178]:
from gensim.models.doc2vec import Doc2Vec,LabeledSentence
sentences = (LabeledSentence(words=[jieba.lcut(dfapp['app_info'][i])], 
                             tags=[dfapp['app_cate'][i]]) for i in range(len(dfapp)))

In [215]:
sentences[0]

TaggedDocument(words='融360   — —   您 随身 的 金融 专家 为什么 要 使用 【 融360 】 ？ 下述 内容 不 吹牛 。 1 .   解决 【 贷款 】 难题 特点 ： -   全面解决 ： 个人消费 贷 、 企业 经营 贷 、 车房 装修 贷 、 出行 旅游 贷 、 学习 进修 贷 。 -   放款 效率 ： 仅 需 一天 即可 放款 。 -   利率 优势 ： 年化 费率 低至 6% ， 还款 期限 3 个 月 起 。 -   服务 优质 ： 我们 深知 您 的 难 和 急 ， 暖心 的 服务 是 我们 理应 为 您 真情 奉献 的 。 2 .   全面 的 【 信用卡 资讯 】 特点 ： -   优惠 资讯 ： 各行 信用卡 优惠 资讯 实时 提供 ， 吃喝玩乐 省 不停 。 -   信用卡 论坛 ： 信用卡 达人 和 小白 都 爱 逛得 专属 论坛 。 3 .   提供 【 金融信息 】 大全 特点 ： -   金融信息 ： 涵盖 各种 办卡 技巧 、 贷款 技巧 、 经济 政策 、 金融 趣谈 。 融360 公司 介绍 融360 ， 成立 于 2011 年 10 月 ， 是 中国 值得 信赖 的 金融 垂直 搜索 平台 。 融360 致力于 为 个人 消费者 和 微小 企业 提供 金融 产品 的 搜索 、 推荐 和 申请 服务 ， 业务范围 涵盖 贷款 、 信用卡 。 此外 ， 融360 还 免费 为 用户 提供 便捷 、 划算 、 安全 的 金融信息 。 融360 目前 已 完成 D 轮 融资 ， 估值 超 10 亿美元 ， 步入 独角兽 俱乐部 。 融360 旗下 简普 科技 2017 年 11 月 在 美国 纽交所 上市 。 联系 我们   — — 【 官方网站 】 http : / / m . rong360 . com 【 官方 微信 公众 号 】 融360 【 客服 电话 】 1010 0360 或 4008 - 905 - 360', tags=['理财'])

In [ ]:
TaggedDocument

In [242]:
# -*- coding: UTF-8 -*-  
import gensim, logging,os
from gensim.models.doc2vec import Doc2Vec,TaggedDocument
import gensim.models.doc2vec

logging.basicConfig(format = '%(asctime)s : %(levelname)s : %(message)s', level = logging.INFO)

sentences = [TaggedDocument(words=jieba.lcut(dfapp['app_info'][i]), 
                             tags=[dfapp['app_cate'][i]]) for i in range(len(dfapp))]

model = Doc2Vec(sentences, size =50, window = 5, min_count=1)

2019-01-14 16:52:20,107 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2019-01-14 16:52:20,109 : INFO : collecting all words and their counts
2019-01-14 16:52:20,109 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2019-01-14 16:52:20,453 : INFO : PROGRESS: at example #10000, processed 1642206 words (4827400/s), 71861 word types, 29 tags
2019-01-14 16:52:20,805 : INFO : PROGRESS: at example #20000, processed 3142663 words (4279008/s), 110304 word types, 30 tags
2019-01-14 16:52:20,891 : INFO : collected 117182 word types and 30 unique tags from a corpus of 22262 examples and 3452337 words
2019-01-14 16:52:20,892 : INFO : Loading a fresh vocabulary
2019-01-14 16:52:21,519 : INFO : effective_min_count=1 retains 117182 unique words (100% of original 117182, drops 0)
2019-01-14 16:52:21,520 : INFO : effective_min_count=1 leaves 3452337 word corpus (100% of original 3452337, drops 0)
2019-01-14 16:52:21,928 : INFO : deleting

In [243]:
model.wv.vocab

{'融360': <gensim.models.keyedvectors.Vocab at 0x64d378d0>,
 ' ': <gensim.models.keyedvectors.Vocab at 0x64d379e8>,
 '—': <gensim.models.keyedvectors.Vocab at 0x64d37ac8>,
 '您': <gensim.models.keyedvectors.Vocab at 0x64d37a20>,
 '随身': <gensim.models.keyedvectors.Vocab at 0x64d37a58>,
 '的': <gensim.models.keyedvectors.Vocab at 0x64d37a90>,
 '金融': <gensim.models.keyedvectors.Vocab at 0x64d379b0>,
 '专家': <gensim.models.keyedvectors.Vocab at 0x64d37b00>,
 '为什么': <gensim.models.keyedvectors.Vocab at 0x64d37b38>,
 '要': <gensim.models.keyedvectors.Vocab at 0x64d37b70>,
 '使用': <gensim.models.keyedvectors.Vocab at 0x64d37ba8>,
 '【': <gensim.models.keyedvectors.Vocab at 0x64d37be0>,
 '】': <gensim.models.keyedvectors.Vocab at 0x64d37c18>,
 '？': <gensim.models.keyedvectors.Vocab at 0x64d37c50>,
 '下述': <gensim.models.keyedvectors.Vocab at 0x64d37c88>,
 '内容': <gensim.models.keyedvectors.Vocab at 0x64d37cc0>,
 '不': <gensim.models.keyedvectors.Vocab at 0x64d37cf8>,
 '吹牛': <gensim.models.keyedvectors.Vo

In [245]:
model.wv.most_similar('支付')

2019-01-14 16:53:28,616 : INFO : precomputing L2-norms of word weight vectors
C:\ProgrameFiles\Anaconda\envs\py3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('收款', 0.8444912433624268),
 ('支付宝', 0.8217622637748718),
 ('银联', 0.8210588097572327),
 ('付款', 0.8048133254051208),
 ('刷卡', 0.7991886734962463),
 ('充值', 0.7850642204284668),
 ('结算', 0.7794612646102905),
 ('担保', 0.7703073620796204),
 ('扫码', 0.768652081489563),
 ('收单', 0.7654709815979004)]

In [244]:
model.docvecs.doctags

{'理财': Doctag(offset=0, word_count=570707, doc_count=3324),
 '社交': Doctag(offset=1, word_count=153349, doc_count=994),
 '系统': Doctag(offset=2, word_count=51008, doc_count=323),
 '工具': Doctag(offset=3, word_count=452592, doc_count=3334),
 '旅游': Doctag(offset=4, word_count=37154, doc_count=208),
 '购物': Doctag(offset=5, word_count=197330, doc_count=1442),
 '安全': Doctag(offset=6, word_count=12829, doc_count=85),
 '娱乐': Doctag(offset=7, word_count=103546, doc_count=728),
 '视频': Doctag(offset=8, word_count=81784, doc_count=471),
 '出行': Doctag(offset=9, word_count=47845, doc_count=318),
 '音乐': Doctag(offset=10, word_count=40540, doc_count=233),
 '通讯': Doctag(offset=11, word_count=23468, doc_count=161),
 '摄影': Doctag(offset=12, word_count=68351, doc_count=356),
 '办公': Doctag(offset=13, word_count=93315, doc_count=646),
 '生活': Doctag(offset=14, word_count=433930, doc_count=3016),
 '网络游戏': Doctag(offset=15, word_count=49368, doc_count=358),
 '休闲益智': Doctag(offset=16, word_count=132778, doc_count

In [256]:
# 计算新文档的向量
inferred_docvec = model.infer_vector(alldocs[5].words)
print('%s:\n %s' % (model, model.docvecs.most_similar([inferred_docvec], topn=10)))


Doc2Vec(dm/m,d50,n5,w5,s0.001,t3):
 [('游戏', 0.6466513276100159), ('飞行射击', 0.6239187717437744), ('经营策略', 0.607236921787262), ('角色扮演', 0.6063746809959412), ('网络游戏', 0.6048875451087952), ('动作冒险', 0.604141891002655), ('体育竞速', 0.585077166557312), ('视频', 0.5832176804542542), ('棋牌中心', 0.579785943031311), ('音乐', 0.5793493986129761)]


In [253]:
dfapp

,app_name,app_info,app_cate,app_score,download_cnt
0,融360,融360 —— 您随身的金融专家为什么要使用【融360】？下述内容不吹牛。1. 解决【贷款】...,理财,4.717574,2.321756e+07
1,微信,1.可以发语音、文字消息、表情、图片、视频30M流量可以收发上千条语音，省电省流量2.朋友圈...,社交,3.533574,7.601503e+09
2,设置,一款精美的屏幕开关手机小部件应用程序，实时装扮你的桌面，喜欢的你还等什么，赶快来下载吧！ 【...,系统,3.000000,8.294300e+04
3,支付宝,【支付宝介绍】蚂蚁金服旗下的支付宝，是以每个人为中心，拥有全球用户9亿的生活服务平台。目前，...,理财,3.583377,9.763013e+08
4,信息,iPhone信息是iPhone风格的消息应用程序。酷像IOS的UI。非常简单易用，就像iph...,工具,2.769231,3.027110e+05
5,QQ,-----QQ•乐在沟通-----√服务超过90%的移动互联网用户√多人视频、文件多端互传，...,社交,3.431373,8.703708e+09
6,指南针,这是一款功能实用，操作简明，性能出众的智能指南针。 它抛去了浮华，洗尽了铅尘。 优化后的...,旅游,4.553153,2.268551e+06
7,手机淘宝,手机淘宝（Android版）是阿里巴巴专为Android手机用户推出的满足其生活消费和线上购...,购物,4.543010,2.653544e+09
8,云服务,乐同步全新改版，这是一个全新的云服务乐同步是一款安全的手机私人数据备份与管理平台一键备份/恢...,安全,3.845070,3.067698e+07
9,WiFi万能钥匙,免费安全WiFi热点，无忧一键连接。无论身处何地，只要打开APP，看到分享热点，点击连接即可...,工具,2.681381,7.740153e+09


In [262]:
model.docvecs.most_similar(0)

[('新闻', 0.503153920173645),
 ('游戏', 0.43816813826560974),
 ('安全', 0.4297948479652405),
 ('网络游戏', 0.4280186891555786),
 ('角色扮演', 0.41850394010543823),
 ('飞行射击', 0.36829185485839844),
 ('动作冒险', 0.3655446171760559),
 ('视频', 0.3646373450756073),
 ('阅读', 0.36164647340774536),
 ('通讯', 0.35929733514785767)]

In [220]:
dfapp

,app_name,app_info,app_cate,app_score,download_cnt
0,融360,融360 —— 您随身的金融专家为什么要使用【融360】？下述内容不吹牛。1. 解决【贷款】...,理财,4.717574,2.321756e+07
1,微信,1.可以发语音、文字消息、表情、图片、视频30M流量可以收发上千条语音，省电省流量2.朋友圈...,社交,3.533574,7.601503e+09
2,设置,一款精美的屏幕开关手机小部件应用程序，实时装扮你的桌面，喜欢的你还等什么，赶快来下载吧！ 【...,系统,3.000000,8.294300e+04
3,支付宝,【支付宝介绍】蚂蚁金服旗下的支付宝，是以每个人为中心，拥有全球用户9亿的生活服务平台。目前，...,理财,3.583377,9.763013e+08
4,信息,iPhone信息是iPhone风格的消息应用程序。酷像IOS的UI。非常简单易用，就像iph...,工具,2.769231,3.027110e+05
5,QQ,-----QQ•乐在沟通-----√服务超过90%的移动互联网用户√多人视频、文件多端互传，...,社交,3.431373,8.703708e+09
6,指南针,这是一款功能实用，操作简明，性能出众的智能指南针。 它抛去了浮华，洗尽了铅尘。 优化后的...,旅游,4.553153,2.268551e+06
7,手机淘宝,手机淘宝（Android版）是阿里巴巴专为Android手机用户推出的满足其生活消费和线上购...,购物,4.543010,2.653544e+09
8,云服务,乐同步全新改版，这是一个全新的云服务乐同步是一款安全的手机私人数据备份与管理平台一键备份/恢...,安全,3.845070,3.067698e+07
9,WiFi万能钥匙,免费安全WiFi热点，无忧一键连接。无论身处何地，只要打开APP，看到分享热点，点击连接即可...,工具,2.681381,7.740153e+09


In [ ]:
# -*- coding: UTF-8 -*-  
import gensim, logging
import os
from gensim.models.doc2vec import Doc2Vec,LabeledSentence
from gensim.models import Doc2Vec
import gensim.models.doc2vec

asin=set()
category=set()
class LabeledLineSentence(object):
    def __init__(self, filename=object):
        self.filename =filename
    def __iter__(self):
        with open(self.filename,'r') as infile:
            data=infile.readlines(); 
            print "length: ", len(data)        
        for uid,line in enumerate(data):
            print "line:",line
            asin.add(line.split("\t")[0])
            print "asin: ",asin
            category.add(line.split("\t")[1])
            yield LabeledSentence(words=line.split("\t")[2].split(" "), tags=[line.split("\t")[0], line.split("\t")[1]])
print 'success'

logging.basicConfig(format = '%(asctime)s : %(levelname)s : %(message)s', level = logging.INFO)
sentences =LabeledLineSentence('product_bpr_test_train.txt')
model = Doc2Vec(sentences, size =50, window = 5, min_count=1)
model.save('product_bpr_model50.txt')
print  'success1'

print "doc2vecs length:", len(model.docvecs)
outid = file('product_bpr_id_vector50.txt', 'w')
outcate = file('product_bpr_cate_vector50.txt', 'w')
for id in asin:
    outid.write(id+"\t")
    for idx,lv in enumerate(model.docvecs[id]):
        outid.write(str(lv)+" ")
    outid.write("\n")
for cate in category:
    outcate.write(cate + '\t')
    for idx,lv in enumerate(model.docvecs[cate]):
        outcate.write(str(lv)+" ")
    outcate.write('\n')
outid.close()
outcate.close()

### FastText示范

FastText 在普通的word2vec基础上考虑了加入N-gram，即短语作为输入特征，因此其在一定意义上考虑了词的顺序。

例如在word2vec中 "我打他" 和"他打我" 是等价的，但在FastText中则加入了短语“我打”，因此考虑了词的顺序。

此外，FastText还考虑了词的拆分，即词根，有些不在字典中的词，在FastText中也能够根据词根相似性计算词向量。

由于有词的拆分这一特性，FastText能够计算任意句子的词向量，只要句子中有些词在字典中即可。

In [11]:
from gensim.test.utils import common_texts
from gensim.models import FastText

model = FastText(common_texts, size=4, window=3, min_count=1, iter=10)

In [33]:
common_texts

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]

In [34]:
model.wv.vocab

{'human': <gensim.models.keyedvectors.Vocab at 0x17f57f28>,
 'interface': <gensim.models.keyedvectors.Vocab at 0x17f57d30>,
 'computer': <gensim.models.keyedvectors.Vocab at 0x17f57da0>,
 'survey': <gensim.models.keyedvectors.Vocab at 0x17f579b0>,
 'user': <gensim.models.keyedvectors.Vocab at 0x17f575c0>,
 'system': <gensim.models.keyedvectors.Vocab at 0x17f57588>,
 'response': <gensim.models.keyedvectors.Vocab at 0x17f57550>,
 'time': <gensim.models.keyedvectors.Vocab at 0x17f57630>,
 'eps': <gensim.models.keyedvectors.Vocab at 0x17f576a0>,
 'trees': <gensim.models.keyedvectors.Vocab at 0x17f579e8>,
 'graph': <gensim.models.keyedvectors.Vocab at 0x17f57470>,
 'minors': <gensim.models.keyedvectors.Vocab at 0x17f574a8>}

In [8]:
model.wv.save_word2vec_format('vector.txt',binary = False,total_vec= None)

In [3]:
from gensim.test.utils import get_tmpfile

fname = get_tmpfile("fasttext.model")

model.save(fname)
model = FastText.load(fname)  # you can continue training with the loaded model!

In [35]:
existent_word = "computer"
existent_word in model.wv.vocab

True

In [36]:
computer_vec = model.wv[existent_word]  # numpy vector of a word
computer_vec

array([ 0.02412866,  0.05822411, -0.00761765,  0.01267011], dtype=float32)

In [37]:
oov_word = "computer-user"
oov_word in model.wv.vocab

False

In [38]:
oov_vec = model.wv[oov_word]  # numpy vector for OOV word

In [39]:
oov_vec

array([ 0.03628856,  0.05038398, -0.00696638,  0.0245754 ], dtype=float32)

In [42]:
# 计算句子的词向量
model.wv['computer']

array([ 0.02412866,  0.05822411, -0.00761765,  0.01267011], dtype=float32)

In [41]:
# 计算句子的词向量
model.wv['computer game is very hard to play']

array([0.01866415, 0.04346466, 0.01676881, 0.01589118], dtype=float32)

In [43]:
model.wv.similarity('computer game is very hard to play','computer')

C:\ProgrameFiles\Anaconda\envs\py3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.8963409

In [44]:
# 使用不在词典中的符号进行分割得到的结果是一样的
model.wv['computer/game/is/very/hard/to/play']

array([0.01866415, 0.04346466, 0.01676881, 0.01589118], dtype=float32)

In [45]:
# 计算句子的相似性
model.wv.similarity('computer game is interesting','user like computer')

0.92745155

In [12]:
similarities = model.wv.most_similar(positive=['computer', 'human'], negative=['interface'])
most_similar = similarities[0]

similarities = model.wv.most_similar_cosmul(positive=['computer', 'human'], negative=['interface'])
most_similar = similarities[0]

not_matching = model.wv.doesnt_match("human computer interface tree".split())
sim_score = model.wv.similarity('computer', 'human')

C:\ProgrameFiles\Anaconda\envs\py3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
